In [6]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
import matplotlib.pyplot as plt
import numpy as np
import dlib
from imutils import face_utils
from pygame import mixer
from scipy.spatial import distance as dist
import mediapipe as mp
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
import matplotlib.pyplot as plt


In [7]:
from playsound import playsound
# for playing wav file

def alert():
    play = playsound("C:/Users/kadem/cdsaml/alert_alarm.wav")


def eye_aspect_ratio(eye):
    p2_minus_p6 = dist.euclidean(eye[1], eye[5])
    p3_minus_p5 = dist.euclidean(eye[2], eye[4])
    p1_minus_p4 = dist.euclidean(eye[0], eye[3])
    ear = (p2_minus_p6 + p3_minus_p5) / (2.0 * p1_minus_p4)
    return ear

def mouth_aspect_ratio(mouth):
	# compute the euclidean distances between the two sets of
	# vertical mouth landmarks (x, y)-coordinates
    A = dist.euclidean(mouth[2], mouth[10]) # 51, 59
    B = dist.euclidean(mouth[4], mouth[8]) # 53, 57

	# compute the euclidean distance between the horizontal
	# mouth landmark (x, y)-coordinates
    C = dist.euclidean(mouth[0], mouth[6]) # 49, 55

	# compute the mouth aspect ratio
    mar = (A + B) / (2.0 * C)

	# return the mouth aspect ratio
    return mar


# FACIAL_LANDMARKS_IDXS = OrderedDict([
# 	("mouth", (48, 68)),
# 	("right_eyebrow", (17, 22)),
# 	("left_eyebrow", (22, 27)),
# 	("right_eye", (36, 42)),
# 	("left_eye", (42, 48)),
# 	("nose", (27, 35)),
# 	("jaw", (0, 17))
# ])

In [8]:
#Defining threshold to classify whether the person is drowsy or not

MOUTH_AR_THRESH = 0.5
EAR_AR_THRESH = 0.2
mouth_count = 0
eye_count = 0
thresh_count = 5


In [9]:
video_capture= cv2.VideoCapture(0)
detector=FaceMeshDetector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detect = dlib.get_frontal_face_detector()
(leftEyeStart, leftEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rightEyeStart, rightEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mouthStart,mouthEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

list_df = []

while True:
        _,frame = video_capture.read()      
        modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
        configFile = "deploy.prototxt.txt"
        net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces = net.forward()
        for i in range(faces.shape[2]):
            confidence = faces[0, 0, i, 2]
            if confidence > 0.5:
                    box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (x, y, x1, y1) = box.astype("int")
                    cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detect(gray)
        area = lambda x,y,w,h:(w-x)*(h-y)
        f = -1
        if len(faces):
            for face in faces:
                x = face.left()
                y = face.top()
                w = face.right()
                h = face.bottom()
                my_area =  area(x,y,w,h)
                if(my_area>f):
                    f = my_area
                    face1 = face       
                    landmarks = predictor(gray, face1)
                    landmarks = face_utils.shape_to_np(landmarks)
                        

#         for n in range(0, 68):
#             (x,y) = landmarks[n]
#             cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)

            leftEye = landmarks[leftEyeStart:leftEyeEnd]
            rightEye = landmarks[rightEyeStart:rightEyeEnd]
        
            mouth = landmarks[mouthStart:mouthEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR + rightEAR) / 2.0
            print(ear)
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            mouthMAR = mouth_aspect_ratio(mouth)
            mar = mouthMAR

            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
            cv2.putText(frame, "MAR: {:.2f}".format(mar), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2,cv2.LINE_AA,False)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2,cv2.LINE_AA,False)
        
            if mar > MOUTH_AR_THRESH and mouth_count < thresh_count:
                mouth_count += 1
                cv2.putText(frame, "Mouth is Open!", (200,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,128,255),2)
                #cv2.putText(frame, "MAR: {:.2f}".format(mar), (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2,cv2.LINE_AA,False)
            
            elif mar > MOUTH_AR_THRESH and mouth_count >= thresh_count:
                alert()
                list_df.append(['Drowsy',ear,mar])
                mouth_count = 0
                cv2.putText(frame, "Drowsy!!", (200,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,128,255),2)
            
            if ear < EAR_AR_THRESH and eye_count < thresh_count:
                cv2.putText(frame, "Eye is closed!", (200,60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,128,255),2)
                eye_count += 1
                
            
            elif ear< EAR_AR_THRESH and eye_count >= thresh_count:
                alert()
                cv2.putText(frame, "Sleepy!", (200,60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,128,255),2)
                list_df.append(['Sleepy',ear,mar])
                eye_count = 0
                
                
            
        cv2.imshow('Video',frame)
        cv2.waitKey(1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            

video_capture.release()
cv2.destroyAllWindows()

0.31491987600322535
0.28777605935203765
0.29773719145970895
0.291540239825968
0.28187289335485466
0.2870043122313309
0.2832478029400136
0.22318749619230746
0.22221480447053799
0.26163187412843625
0.26122298275594996
0.2620850924465564
0.26759727859278337
0.2778865708145333
0.2553116728289208
0.24445236675984408
0.2242627793972226
0.20609218247385913
0.185113803722196
0.20123783335606044
0.18826950066420914
0.1817609383510348
0.178498436547312
0.2111250211773697
0.18289765057561397
0.19990099168865305
0.29274096308572906
0.29280203208778105
0.291489962859524
0.2989268132152749
0.2921739130434783
0.2978808195361129
0.28090567331957284
0.3105593287120122
0.30288959103615104
0.2811709788979976
0.3102787284449374
0.2984768393895746
0.29897904331195957
0.29220007311702645
0.2919184191881248
0.2915844239621318
0.3057318680939508
0.3054404754837641
0.2607731265928682
0.22865331747512965
0.20439162353230017
0.18751564297708154
0.22385152623118565
0.19884991198104654
0.24723298224649276
0.272450

In [11]:
#Writing state, ear, mar into csv

df = pd.DataFrame(list_df,columns = ['State','EAR','MAR'])
df

,State,EAR,MAR
0,Sleepy,0.199901,0.377756
1,Sleepy,0.195279,0.426497
2,Drowsy,0.244568,0.500108


In [12]:
df.describe()

,EAR,MAR
count,3.000000,3.000000
mean,0.213249,0.434787
std,0.027221,0.061596
min,0.195279,0.377756
25%,0.197590,0.402126
50%,0.199901,0.426497
75%,0.222235,0.463302
max,0.244568,0.500108
